## Exploratory Data Analysis and Data Cleaning

In [41]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [42]:
data_folder = "..\\data\\raw"
os.listdir(data_folder)

['Testing.csv', 'Testing2.csv', 'Training.csv']

In [52]:
#load the data

training_data = pd.read_csv(os.path.join(data_folder, "Training.csv"), index_col=False)
testing_data = pd.read_csv(os.path.join(data_folder, "Testing2.csv"), index_col=False)

training_data.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [44]:
print(training_data.info())
training_data.dtypes

<class 'pandas.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 133 entries, itching to prognosis
dtypes: int64(132), str(1)
memory usage: 5.0 MB
None


itching                 int64
skin_rash               int64
nodal_skin_eruptions    int64
continuous_sneezing     int64
shivering               int64
                        ...  
inflammatory_nails      int64
blister                 int64
red_sore_around_nose    int64
yellow_crust_ooze       int64
prognosis                 str
Length: 133, dtype: object

In [53]:
#check for duplicates
training_data.duplicated().sum()

np.int64(4616)

In [54]:
print(f'Percentage of duplicates: {training_data.duplicated().sum() / len(training_data):.2%}')

Percentage of duplicates: 93.82%


In [46]:
#change data types to categorical for all columns except the target variable
training_data.columns[:-1]
for col in training_data.columns[:-1]:
    training_data[col] = training_data[col].astype('category')

training_data.dtypes

itching                 category
skin_rash               category
nodal_skin_eruptions    category
continuous_sneezing     category
shivering               category
                          ...   
inflammatory_nails      category
blister                 category
red_sore_around_nose    category
yellow_crust_ooze       category
prognosis                    str
Length: 133, dtype: object